# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize  
from string import punctuation

import adagram
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from razdel import tokenize as razdel_tokenize
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import *


In [21]:
wordnet_lemmatizer = WordNetLemmatizer()

def normalize(text):
    text = " ".join(text)
    tokens = word_tokenize(text)
    tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens if word not in punctuation]   
    return tokens

In [7]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0

    normalized = normalize(sentence)
    if word in normalized:
        word_ind = normalized.index(word)
        normalized = normalized[:word_ind] + normalized[word_ind:]
    for i, syns in enumerate(wn.synsets(word)):
        signature = normalize(syns.definition())
        overlap = len(set(signature) & set(normalized))
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i

    return bestsense

Работать функция должна как-то так:

In [8]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time'.split()) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

0

In [9]:
wn.synsets('day')

[Synset('day.n.01'),
 Synset('day.n.02'),
 Synset('day.n.03'),
 Synset('day.n.04'),
 Synset('day.n.05'),
 Synset('day.n.06'),
 Synset('day.n.07'),
 Synset('sidereal_day.n.01'),
 Synset('day.n.09'),
 Synset('day.n.10')]

In [10]:
# с помощью этого индекса достаем нужный синсет
wn.synsets('day')[1].definition()

'some point or period in time'

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [11]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [12]:
corpus_wsd = corpus_wsd[:2000]

In [57]:
res_arr = []

for sent in corpus_wsd:
    sentence = [word[1] for word in sent]
    for word in sent:
        if word[0] != '':
            lesk_index = lesk(word[1], sentence)
            if wn.synsets(word[1])[lesk_index] == wn.lemma_from_key(word[0]).synset():
                res_arr.append(1)
            else:
                res_arr.append(0)

In [104]:
f'accuracy: {sum(res_arr) / len(res_arr)}'

'accuracy: 0.5438470464717909'

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [75]:
import urllib.request
url = 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
filename = 'all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
urllib.request.urlretrieve(url, filename)

('all.a010.p10.d300.w5.m100.nonorm.slim.joblib',
 <http.client.HTTPMessage at 0x24e93a5d3d0>)

In [76]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [77]:
df = pd.read_csv('train.csv', sep='\t')

In [78]:
def normalize(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [79]:
def get_embedding_adagram(text, model, window, dim):
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 

        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [100]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [101]:
models = [KMeans(3, max_iter=200),
          DBSCAN(min_samples=2, leaf_size=10),
          AffinityPropagation(damping=0.6, verbose=True)]

In [ ]:
dim = 300
window = 5
models_res = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), dim))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding_adagram(text, vm, window, dim)
    
    for m in models:
        m.fit(X)
        labels = np.array(m.labels_) + 1
        models_res.append(np.mean(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels)))


### ARI value:

In [107]:
print('KMeans: ', models_res[0])
print('DBSCAN: ', models_res[1])
print('AffinityPropagation: ', models_res[2])

KMeans:  0.6274375641464249
DBSCAN:  0.07154885612190416
AffinityPropagation:  0.11720466285352386


Лучший результат показывает алгоритм KMeans